<h1>Content Based Recommendation System

<h3>Including Libraries

In [223]:
import numpy as np 
import pandas as pd
import ast
from pandas.api.types import is_numeric_dtype
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

<h3> task 2<br>Loading Dataset

In [224]:
movies=pd.read_csv("movies_metadata.csv",low_memory=False)
links=pd.read_csv('links_small.csv',low_memory=False)
links.head()
#this dataset is a collection of movieid and their corresponding imdb an tmdb ids

movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0

<h2>Code From Genre based Recommendation system

In [225]:

#fixing incorrect data with ligitimate data in budget dataset
movies[movies['budget']=='/ff9qCepilowshEtG2GYWwzt2bs4.jpg']#we get that it is at loc 19730
movies['budget'].iloc[19730]="0"
movies[movies['budget']=='/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg']#we get that it is at loc 29503
movies['budget'].iloc[29503]="0"
movies[movies['budget']=='/zaSf5OG7V8X8gqFvly88zDdRm46.jpg']#we get that it is at loc 35587
movies['budget'].iloc[35587]="0"
movies['budget']=movies['budget'].astype(int)

C:\Users\sudhir gupta\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [226]:
#replacing all the columns with value 0 to mean of data
mean=np.mean(movies['budget'])
movies['budget']=movies['budget'].apply(lambda x:mean if x==0 else x)
    
        

In [227]:

#finding non null values of vote_count 
vote_count=movies['vote_count']
vote_count=vote_count.dropna()


#finding non null values of vote_average
vote_average=movies['vote_average']
vote_average=vote_average.dropna()


In [228]:
movies['genres']=movies['genres'].apply(ast.literal_eval)
#checking if all the values are of the form list if not replacing the data with empty list
movies['genres']=movies['genres'].apply(lambda x:x if isinstance(x,list) else [])


In [229]:

#converting the data into list of lists  (contained in upper ) in which each inner list contains all genre
#corresponding to a single movie 
upper=[]

for i in movies['genres']:
    inner=[]
    for j in i:
        inner.append(j['name'])
    upper.append(inner)

In [230]:
#converting the dataset into 1NF form
md=pd.DataFrame(upper).stack().reset_index(level=1)
del md['level_1']
md.columns=['genre']

In [231]:
#converting the release date column of movies into sting
movies['release_date']=movies['release_date'].astype(str)

In [232]:
#Adding the year column in md dataframe
year=movies['release_date'].apply(lambda x: x.split('-')[0])
md.head()

genre
0  Animation
0     Comedy
0     Family
1  Adventure
1    Fantasy

In [233]:
temp_list=movies.columns
temp_list=list(temp_list)
temp_list.remove('genres')
for columns in temp_list:
    temp_list1=movies[columns].tolist()
    
    temp_list2=[]
    for i in md.index:
        temp_list2.append(temp_list1[i])
    md[columns]=temp_list2


<h3>task 3. <br>Analysing the New Dataset

In [234]:
links.info()
#this depicts all columns are having 0 null values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
movieId    9125 non-null int64
imdbId     9125 non-null int64
tmdbId     9112 non-null float64
dtypes: float64(1), int64(2)
memory usage: 213.9 KB


In [235]:
#basic statistical analysis of dataset
links.describe()

movieId        imdbId         tmdbId
count    9125.000000  9.125000e+03    9112.000000
mean    31123.291836  4.798244e+05   39104.545544
std     40782.633604  7.431774e+05   62814.519801
min         1.000000  4.170000e+02       2.000000
25%      2850.000000  8.884600e+04    9451.750000
50%      6290.000000  1.197780e+05   15852.000000
75%     56274.000000  4.284410e+05   39160.500000
max    164979.000000  5.794766e+06  416437.000000

In [236]:
links.shape

(9125, 3)

<h3>task 4. <br>Data Cleaning

In [237]:
#taking all the notnull values of tmbdid 
links_small=links[links['tmdbId'].notnull()]
links_small=links_small['tmdbId']

In [238]:
#converting the column into int
links_small=links_small.astype(int)

In [239]:
#checking if the data in md dataframe created in genre system is of numeric type or not
is_numeric_dtype(md['id'])

False

In [240]:
md=md.drop(19730)#19730
md=md.drop(29503)#29503
md=md.drop(35587)#35587
md['id']=md['id'].astype(int)


<h3>Task 5<br>Data Wrangling

In [241]:
#getting a list of unique id from links_small dataset to compare it with md dataframe
id_list=links_small.tolist()
id_list=set(id_list)
id_list=list(id_list)

In [242]:
#comparing id column of links_small and md dataframe
filter1=md['id'].isin(id_list)

In [243]:
#putting the filterred data in md_new dataframe
md_new=md[filter1]

In [263]:
#since the dataset is v large taking only a fraction of the dataset to work upon 
md_new_sample=md_new.sample(frac=0.15,random_state=20)

In [264]:
#dropping all null rows in tagline column
temp_list=md_new_sample[md_new_sample['tagline'].isna()]['tagline'].index.tolist()
md_new_sample=md_new_sample.drop(temp_list)

In [265]:
#combining the data from tagline and overview column and making a new column called description in md_new_sample dataframe
md_new_sample['description']=md_new_sample['tagline']+md_new_sample['overview']
md_new_sample['description'].iloc[0]

"Don't believe the fairy tale.The untold story of Disney's most iconic villain from the 1959 classic 'Sleeping Beauty'. A beautiful, pure-hearted young woman, Maleficent has an idyllic life growing up in a peaceable forest kingdom, until one day when an invading army threatens the harmony of the land.  Maleficent rises to be the land's fiercest protector, but she ultimately suffers a ruthless betrayal – an act that begins to turn her heart into stone. Bent on revenge, Maleficent faces an epic battle with the invading King's successor and, as a result, places a curse upon his newborn infant Aurora. As the child grows, Maleficent realizes that Aurora holds the key to peace in the kingdom - and to Maleficent's true happiness as well."

<h3>Task 6<br>Preparing data for Content Based Recommender

In [266]:
#initializing tf-idf vectorizer
tfidf=TfidfVectorizer(min_df=1,stop_words='english',ngram_range=(1,2))

In [267]:
#applying tfidf vectorizer to convert words to vectors
matrix=tfidf.fit_transform(md_new_sample['description'])


In [280]:
#converting the matrix formed by applying tfidf into 2d array 
array=matrix.toarray()
array.shape

(2684, 83821)

In [269]:
#appying cosine similarity principle 
similar=cosine_similarity(array,array)

In [270]:
#resetting the index to get a new proper indexing
md_new_sample=md_new_sample.reset_index()


In [271]:
#saving titles of movies in a new pandas series
title=md_new_sample['title']

In [274]:
#creating a new pandas series with titles and proper indexes
indices=pd.Series(np.arange(2684),index=title)


<h3>Task 7. Building a Content Based recommendation system

In [275]:
#function which takes movie name as input and recommends top 10 movies to the user
def recommendation(name):
    index=indices.loc[name]
    similarity=list(enumerate(similar[index]))
    a=sorted(similarity, key=lambda x: x[1],reverse=True)
    a=a[1:11]
    index1=[i[0] for i in a]
    print(indices.iloc[index1])
    

<h2>I have done everything correct as per my knowledge but I dont know why there are some 
<br>redundant entries of movie titles present in recommendation system in the last part 

In [276]:
#Top 10 recommendation for the movie avatar
recommendation(r"Avatar")

title
Lara Croft Tomb Raider: The Cradle of Life    2208
The Matrix                                    2237
Hellraiser: Bloodline                          173
Supernova                                     1865
The Hidden                                    1980
Green Zone                                    1772
Heaven Knows, Mr. Allison                     1225
Transformers: Age of Extinction                 28
Transformers: Age of Extinction               1847
Rules of Engagement                           1405
dtype: int32


In [278]:
#my dataset (15% sample)does not contain the movie Godfather so taking movie the matrix
recommendation(r"The Matrix")

title
Supernova                1865
Hackers                   433
Hackers                   813
Avatar                    149
Hellraiser: Bloodline     173
Commando                 1882
Transcendence             976
The Matrix Reloaded       278
Gladiator 1992            463
Alfie                    1570
dtype: int32
